In [10]:
import tensorflow as tf
import numpy      as np

import directoryFunctions
import pathlib
import data
import time

from tensorflow.keras.optimizers  import Adam
from tensorflow.keras.callbacks   import CSVLogger, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.losses      import CategoricalCrossentropy
from tensorflow.keras.layers      import Dense, Dropout, LSTM
from tensorflow.keras             import Sequential

In [2]:
def getSequences(datasetType):
    sequences = []
    labels    = []
    for dataRow in dataObj.data[:]:
        if(datasetType == dataRow[0]):
            sequencePath = pathlib.Path(r"/home/jupyter/action-recognition/Sequences")/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence.npy")
            sequence     = np.load(sequencePath)
            sequences.append(sequence)
            label = dataObj.getClassIndex(dataRow[1])
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.uint8)

In [3]:
def getDataset(sequences, labels):
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    return dataset

In [4]:
def prepareTrainDataset(dataset, cache, shuffleBufferSize):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size = shuffleBufferSize)
    # Repeat forever
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [5]:
def getModel():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(256, input_shape=(None, 2048)))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(numClasses, activation = 'softmax')) # change 3 to numClasses
    
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
                  loss      = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [6]:
def trainModel(model, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, callbacks):
    history = model.fit(trainDataset, 
                        epochs = epochs,
                        validation_data  = validationDataset,
                        steps_per_epoch  = steps_per_epoch,
                        validation_steps = validation_steps,
                        callbacks = callbacks)
    return model, history

In [7]:
dataObj    = data.Data()
numClasses = dataObj.numClasses
AUTOTUNE   = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [11]:
def main():
    epochs = 5
    
    cachePath     = pathlib.Path(r"./Cache")
    cacheFilePath = cachePath/'trainRNNDatasetCache'
    directoryFunctions.createDirectory(cachePath)
    
    rootPath = pathlib.Path(r"/home/jupyter/action-recognition/")
    rnnCallbacksDirectory = rootPath/'Callbacks'/'RNN'/f'{numClasses}'
    
    trainSeqCount      = dataObj.getDatasetCount('Train')
    validationSeqCount = dataObj.getDatasetCount('Validation')
    
    trainSequences, trainLabels           = getSequences('Train')
    validationSequences, validationLabels = getSequences('Validation')
    
    trainDataset      = getDataset(trainSequences, trainLabels)
    validationDataset = getDataset(validationSequences, validationLabels)
    
    trainDataset      = prepareTrainDataset(trainDataset, str(cacheFilePath), trainSeqCount)
    validationDataset = validationDataset.batch(BATCH_SIZE)
    
    steps_per_epoch  = np.ceil(trainSeqCount/BATCH_SIZE)
    validation_steps = np.ceil(validationSeqCount/BATCH_SIZE)
    
    modelCheckpointDirectory = rnnCallbacksDirectory/'ModelCheckpoint'
    tensorboardDirectory     = rnnCallbacksDirectory/'Tensorboard'
    csvLoggerDirectory       = rnnCallbacksDirectory/'CSVLogger'
    
    directoryFunctions.createDirectory(csvLoggerDirectory)
    
    modelCheckpoint = ModelCheckpoint(filepath       = str(modelCheckpointDirectory/'RNN_{epoch:03d}_{val_loss:.2f}'),
                                      save_best_only = True)
    tensorboard     = TensorBoard(log_dir = str(tensorboardDirectory/f'{int(time.time())}'))
    csvLogger       = CSVLogger(str(csvLoggerDirectory/f'{int(time.time())}.log'))
    earlyStopping   = EarlyStopping(monitor = 'val_loss', patience = 5)
    callbacks       = [modelCheckpoint, tensorboard, csvLogger, earlyStopping]
    
    model = getModel()
    trained_model, history = trainModel(model, epochs, 
                                        trainDataset, validationDataset, 
                                        steps_per_epoch, validation_steps, callbacks)
    
    directoryFunctions.removeDirectory(cachePath)

In [12]:
main()

Train for 8.0 steps, validate for 3.0 steps
Epoch 1/5
6/8 [=====================>........] - ETA: 1s - loss: 3.2488 - accuracy: 0.2188WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /home/jupyter/action-recognition/Callbacks/RNN/3/ModelCheckpoint/RNN_001_3.74/assets
8/8 [==============================] - 8s 952ms/step - loss: 3.2867 - accuracy: 0.2383 - val_loss: 3.7426 - val_accuracy: 0.5181
Epoch 2/5
8/8 [==============================] - 4s 495ms/step - loss: 3.1549 - accuracy: 0.6133 - val_loss: 3.7424 - val_accuracy: 0.2410
Epoch 3/5
8/8 [==============================] - 4s 446ms/step - loss: 3.3090 - accuracy: 0.2656 - val_loss: 3.7423 